# Extract video from sequence of images


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os 
os.environ['MPLCONFIGDIR'] = '/myhome'
from datetime import datetime
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), os.path.pardir)))
sys.path.append("..")
from data import BavarianCrops, BreizhCrops, SustainbenchCrops, ModisCDL
from models.earlyrnn import EarlyRNN
from models.daily_earlyrnn import DailyEarlyRNN
import torch
from utils.losses.early_reward_loss import EarlyRewardLoss
from utils.losses.stopping_time_proximity_loss import StoppingTimeProximityLoss, sample_three_uniform_numbers
from utils.losses.daily_reward_loss import DailyRewardLoss
import pandas as pd
import pickle 
import wandb
import json
from utils.plots import plot_label_distribution_datasets, boxplot_stopping_times, plot_timestamps_left_per_class
from utils.doy import get_doys_dict_test, get_doy_stop
from utils.helpers_training import parse_args
from utils.helpers_testing import test_dataset, get_test_stats
import matplotlib.pyplot as plt
from utils.plots_test import plot_doy_prob, plot_all_doy_probs, plot_confusion_matrix, plot_probability_stopping, \
    plot_fig_class_prob_wrt_time, plot_fig_class_prob_wrt_time_one_sample, plot_fig_class_prob_wrt_time_with_mus
from models.model_helpers import count_parameters
from utils.extract_mu import extract_mu_thresh
from IPython.display import Image
from IPython.display import Video
from utils.results_analysis.extract_video import download_images, add_files_to_images, save_video


## Download the model from wandb 

In [3]:
api = wandb.Api()
entity, project, sweep = "aurenore", "MasterThesis", "x6ma4kpb"
runs = api.runs(entity + "/" + project)

summary_list, config_list, name_list, sweep_list, start_date_list = [], [], [], [], []
for run in runs:
    # .summary contains output keys/values for
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)
    
    if run.sweep:
        sweep_list.append(run.sweep.name)
    else:
        sweep_list.append(None)
        
    start_date_list.append(run.createdAt)


runs_df = pd.DataFrame(
    {"summary": summary_list, "config": config_list, "name": name_list, "sweep": sweep_list, "start_date": start_date_list}
)
runs_df

,summary,config,name,sweep,start_date
0,"{'loss': {'testloss': 7.443081855773926, 'trai...","{'mu': 150, 'loss': 'daily_reward_lin_regr', '...",soft-sweep-106,daily_reward_lin_regr_with_mus_v2,2024-06-07T18:30:04
1,"{'_runtime': 2773.031559944153, 'conf_mat_tabl...","{'mu': 150, 'loss': 'daily_reward_lin_regr', '...",vibrant-sweep-105,daily_reward_lin_regr_with_mus_v2,2024-06-07T18:16:18
2,"{'epoch': 100, 'precision': 0.5517589301461332...","{'mu': 150, 'loss': 'daily_reward_lin_regr', '...",winter-sweep-104,daily_reward_lin_regr_with_mus_v2,2024-06-07T18:15:55
3,"{'_wandb': {'runtime': 3344}, 'alphas': [0.800...","{'mu': 150, 'loss': 'daily_reward_lin_regr', '...",dainty-sweep-103,daily_reward_lin_regr_with_mus_v2,2024-06-07T18:00:02
4,"{'kappa': 0.5321212218630831, '_runtime': 3464...","{'mu': 150, 'loss': 'daily_reward_lin_regr', '...",confused-sweep-102,daily_reward_lin_regr_with_mus_v2,2024-06-07T17:32:01
...,...,...,...,...,...
750,"{'_wandb': {'runtime': 21797}, 'recall': 0.527...","{'alpha': 0.5, 'device': 'cuda', 'epochs': 100...",TempCNN-seqlength150,None,2024-03-28T15:16:29
751,"{'_step': 46, 'fscore': 0.5579019448823247, 'b...","{'alpha': 0.5, 'device': 'cpu', 'epochs': 100,...",LSTM-seqlength150,None,2024-03-28T15:15:39
752,"{'precision': 0.5533549974802285, 'classificat...","{'alpha': 0.5, 'device': 'cuda', 'epochs': 100...",TempCNN-seqlength70,None,2024-03-28T13:54:08
753,"{'_timestamp': 1711570628.1319556, 'conf_mat_t...","{'alpha': 0.5, 'device': 'cuda', 'epochs': 100...",ELECTS-LSTM-hiddendims=32,None,2024-03-27T14:55:09


get the runs with certain sweep name, and created after a certain date

In [4]:
sweep_name = "daily_reward_lin_regr_with_mus_v2"
# get the best run
df = runs_df[runs_df.sweep == sweep_name]


Pick the run with the highest harmonic mean 

In [5]:
# get the run with the highest harmonic mean
metric = "harmonic_mean"
chosen_run_idx = df.summary.apply(lambda x: x[metric]).idxmax()
chosen_run = runs[chosen_run_idx]
chosen_run_config = chosen_run.config
print("chosen run: ", chosen_run.name)
print(f"with {metric}: ", chosen_run.summary._json_dict[metric])

chosen run:  stoic-sweep-70
with harmonic_mean:  0.6690256704589369


## Sequence of image to video 

In [6]:
path = "media/images/"
name_image = "class_probabilities_wrt_time"
local_path = f"../coding_test/media/{chosen_run.name}"
download_images(path, name_image, chosen_run, local_path)

Downloaded media/images/class_probabilities_wrt_time_11_a516e041d0c674491135.png
Downloaded media/images/class_probabilities_wrt_time_1_02dd425f06d7caf5e836.png
Downloaded media/images/class_probabilities_wrt_time_21_05a9c3ec0e8860693949.png
Downloaded media/images/class_probabilities_wrt_time_31_4afedcf4adc965c2f7a1.png
Downloaded media/images/class_probabilities_wrt_time_41_307fd7d9f5701cf2294e.png
Downloaded media/images/class_probabilities_wrt_time_51_c589e341b8bf1556cf1c.png
Downloaded media/images/class_probabilities_wrt_time_61_7e64eb07b0ba5b2819bd.png
Downloaded media/images/class_probabilities_wrt_time_71_892606d43c00475c8181.png
Downloaded media/images/class_probabilities_wrt_time_81_9b4c2beaf4d94f6c74d2.png
Downloaded media/images/class_probabilities_wrt_time_91_40a6401e4648816e7d69.png
Total downloaded: 10 files.


In [7]:
images_directory = os.path.join(local_path, "media", "images")
images = add_files_to_images(images_directory, name_image)
video_path = save_video(images_directory, images)

imags_paths:  ['../coding_test/media/stoic-sweep-70\\media\\images\\class_probabilities_wrt_time_1_02dd425f06d7caf5e836.png', '../coding_test/media/stoic-sweep-70\\media\\images\\class_probabilities_wrt_time_11_a516e041d0c674491135.png', '../coding_test/media/stoic-sweep-70\\media\\images\\class_probabilities_wrt_time_21_05a9c3ec0e8860693949.png', '../coding_test/media/stoic-sweep-70\\media\\images\\class_probabilities_wrt_time_31_4afedcf4adc965c2f7a1.png', '../coding_test/media/stoic-sweep-70\\media\\images\\class_probabilities_wrt_time_41_307fd7d9f5701cf2294e.png', '../coding_test/media/stoic-sweep-70\\media\\images\\class_probabilities_wrt_time_51_c589e341b8bf1556cf1c.png', '../coding_test/media/stoic-sweep-70\\media\\images\\class_probabilities_wrt_time_61_7e64eb07b0ba5b2819bd.png', '../coding_test/media/stoic-sweep-70\\media\\images\\class_probabilities_wrt_time_71_892606d43c00475c8181.png', '../coding_test/media/stoic-sweep-70\\media\\images\\class_probabilities_wrt_time_81_9b4c2

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1500, 4900) to (1504, 4912) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


10 images added to the video.
Video saved at ../coding_test/media/stoic-sweep-70\media\images\class_probability_wrt_time.mp4


In [8]:
# play video in the notebook
Video(video_path)


let's do the same for boxplots

In [9]:
path = "media/images/"
name_image = "boxplot"
local_path = f"../coding_test/media/{chosen_run.name}"
download_images(path, name_image, chosen_run, local_path)

images_directory = os.path.join(local_path, "media", "images")
images = add_files_to_images(images_directory, name_image)
video_path = save_video(images_directory, images, name_image+"_video.mp4")

Video(video_path)


Downloaded media/images/boxplot_11_fb7bf87b3c90b6244ef1.png
Downloaded media/images/boxplot_1_fb7bf87b3c90b6244ef1.png
Downloaded media/images/boxplot_21_fb7bf87b3c90b6244ef1.png
Downloaded media/images/boxplot_31_fb7bf87b3c90b6244ef1.png
Downloaded media/images/boxplot_41_06b7dc0eefab886078a8.png
Downloaded media/images/boxplot_51_47b2382713fbf95300d9.png
Downloaded media/images/boxplot_61_3a905e0060713c790587.png
Downloaded media/images/boxplot_71_087668b7de38a533c6ee.png
Downloaded media/images/boxplot_81_01cb4c969b9e120c657e.png
Downloaded media/images/boxplot_91_c381db22495a876a135f.png


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1500, 700) to (1504, 704) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Total downloaded: 10 files.
imags_paths:  ['../coding_test/media/stoic-sweep-70\\media\\images\\boxplot_1_fb7bf87b3c90b6244ef1.png', '../coding_test/media/stoic-sweep-70\\media\\images\\boxplot_11_fb7bf87b3c90b6244ef1.png', '../coding_test/media/stoic-sweep-70\\media\\images\\boxplot_21_fb7bf87b3c90b6244ef1.png', '../coding_test/media/stoic-sweep-70\\media\\images\\boxplot_31_fb7bf87b3c90b6244ef1.png', '../coding_test/media/stoic-sweep-70\\media\\images\\boxplot_41_06b7dc0eefab886078a8.png', '../coding_test/media/stoic-sweep-70\\media\\images\\boxplot_51_47b2382713fbf95300d9.png', '../coding_test/media/stoic-sweep-70\\media\\images\\boxplot_61_3a905e0060713c790587.png', '../coding_test/media/stoic-sweep-70\\media\\images\\boxplot_71_087668b7de38a533c6ee.png', '../coding_test/media/stoic-sweep-70\\media\\images\\boxplot_81_01cb4c969b9e120c657e.png', '../coding_test/media/stoic-sweep-70\\media\\images\\boxplot_91_c381db22495a876a135f.png']
10 images added to the video.
Video saved at ../